In [40]:
import charts
import pymongo

In [45]:
client = pymongo.MongoClient('localhost', 27017)
ceshi = client['ganji']
item_info = ceshi['item_info']

在这里需要将价格改为float型写入进去  
先看看是啥样子

In [50]:
for i in item_info.find().limit(100):
    print(i['price'])

450 元
1500 元
260 元
1500
面议
800 元
1100
面议
900
380 元
1300 元
11
1100
1600
1500 元
50
60
3000
110
1500 元
600 元
69
300
3000
200 元
600 元
1300 元
80
200
750
150 元
8
390 元
700 元
3800
4600
1500
3000
280 元
500 元
550 元
950
550
1400 元
800 元
700 元
650
2000
450
面议
1250
2100
100 元
200 元
450
900
600 元
80 元
200
1600
800 元
1000
750 元
50
380
500
4000 元
2400 元
800 元
4000
50
8000 元
1288
1000
300 元
600 元
500 元
320
380 元
400
360 元
1000 元
3600
350 元
12
600
4000 元
2100
500
330 元
3950
360 元
900 元
6500
1500
1000
950 元
800 元
4000
350 元


可以先分割，然后把前面的转换成float，后面的去掉，遇到非数字直接改为0

In [58]:
for i in item_info.find():
    try:
        price = (float(i['price'].split()[0])) 
    except ValueError:
        price = 0
    item_info.update_one({'_id':i['_id']},{'$set':{'price':price}})#更新数据库的内容

In [60]:
for i in item_info.find().limit(100):
    print(i['price'])

450.0
1500.0
260.0
1500.0
0
800.0
1100.0
0
900.0
380.0
1300.0
11.0
1100.0
1600.0
1500.0
50.0
60.0
3000.0
110.0
1500.0
600.0
69.0
300.0
3000.0
200.0
600.0
1300.0
80.0
200.0
750.0
150.0
8.0
390.0
700.0
3800.0
4600.0
1500.0
3000.0
280.0
500.0
550.0
950.0
550.0
1400.0
800.0
700.0
650.0
2000.0
450.0
0
1250.0
2100.0
100.0
200.0
450.0
900.0
600.0
80.0
200.0
1600.0
800.0
1000.0
750.0
50.0
380.0
500.0
4000.0
2400.0
800.0
4000.0
50.0
8000.0
1288.0
1000.0
300.0
600.0
500.0
320.0
380.0
400.0
360.0
1000.0
3600.0
350.0
12.0
600.0
4000.0
2100.0
500.0
330.0
3950.0
360.0
900.0
6500.0
1500.0
1000.0
950.0
800.0
4000.0
350.0


# 问题：某段时间内，北京各个城区发帖数量的 top3 类目
## 已知：  
1.某个时间段，例：2015.12.25~2015.12.27  
2.北京各个城区，例：朝阳  
## 求解：  
1.发帖数量 top3  
## 所需数据结构：  
1.原始 :series = [{'name': 'name','data': [100]},{'name': 'name','data': [100]}]  
2.实际上：{name:类目，data:发帖量}  
3.目标：{'_id': ['北京二手家电'], 'counts': 175}  

In [29]:
def data_gen1(date1,date2,area,limit):
    pipeline = [
        {'$match':{'$and':[{'pub_date':{'$gte':date1,'$lte':date2}},{'area':{'$all':[area]}}]}},
        {'$group':{'_id':{'$slice':['$cates',2,1]},'counts':{'$sum':1}}},
        {'$sort':{'counts':-1}},
        {'$limit':limit}
    ]
    for i in item_info.aggregate(pipeline):
        yield{
            'name':i['_id'][0],
            'data':[i['counts']],
            'type': 'column'
        }
for i in data_gen1('2015.12.25','2015.12.27','朝阳',5):
    print(i)


{'name': '北京二手家电', 'data': [175], 'type': 'column'}
{'name': '北京二手服装/鞋帽/箱包', 'data': [105], 'type': 'column'}
{'name': '北京二手办公用品/设备', 'data': [86], 'type': 'column'}
{'name': '北京二手母婴/儿童用品', 'data': [81], 'type': 'column'}
{'name': '北京二手文体/户外/乐器', 'data': [70], 'type': 'column'}


In [30]:
options = {
    'chart'   : {'zoomType':'xy'},
    'title'   : {'text': '发帖量统计'},
    'subtitle': {'text': '可视化统计图表'},
    'yAxis'   : {'title': {'text': '数量'}}
    }

series =[i for i in data_gen1('2015.12.25','2015.12.27','朝阳',5)]

charts.plot(series,options=options,show='inline')
#series

# 问题：某段时间内各大类目中成色对应的平均价格  
## 已知：  
1.某个时间段，例：2015.12.25~2015.12.27  
2.各大类目，例：北京二手手机  
3.单个物品价格，例：某品牌二手电视，价格1000元  
## 求解：  
1.平均价格  
## 所需数据结构：  
1.原始：series = ['data1'，'data2'，'data3']  
2.实际上：[全新对应的价格，9成新对应的价格，8成新对应的价格，7成新对应的价格]  
3.目标：{'avg_price': 300.0, '_id': '7成新及以下'}

In [67]:
def data_gen2(date1,date2,cate):
    pipeline = [
        {'$match': {
            '$and': [{
                'pub_date': {
                    '$gte': date1,
                    '$lte': date2}}, 
                {
                'cates': {
                    '$all': [cate]}}, 
                {
                'look': {
                    '$nin': ['-']}}, 
                {
                'price': {
                    '$gte': 0}}]}},
        {'$group':{'_id':'$look','ave_price':{'$avg':'$price'}}},
        {'$sort':{'ave_price':1}}
    ]    
    for i in item_info.aggregate(pipeline):
        yield i['ave_price']
for i in data_gen2('2015.12.25','2015.12.27','北京二手手机'):
    print(i)


220.0
300.0
730.0
1187.5151515151515
1626.923076923077
1704.5
1849.7894736842106


In [71]:
data = [i for i in data_gen2('2015.12.25','2016.01.10','北京二手手机')]
options = {
    'title': {'text': '新旧-价格'},
    'xAxis'   : {'categories': ['报废机/尸体','7成新及以下','8成新','9成新','95成新','99成新', '全新']},
    'yAxis'   : {'title': {'text': '价格'}},
    
}
charts.plot(data,show='inline', options=options)